# IMPORTS AND API_KEY

In [1]:
import requests
import time
import mysql.connector
import test_config
api_key = '91011d12ab785b50295a8f31e8f78a6b'
db_name = 'movies'

# CREATE DATABASE

In [2]:
cnx = mysql.connector.connect(
        host = test_config.host,
        user = test_config.user,
        password = test_config.password
)
cursor = cnx.cursor(buffered=True)

In [3]:
cursor.execute("""CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'""".format(db_name))
cnx.close()

# CREATE TABLE(S)

"create_table" creates tables into your database. Takes a query and database as variables.

In [7]:
def create_tmdb_table(table_name, db_name):
    cnx = mysql.connector.connect(
        host = test_config.host,
        user = test_config.user,
        password = test_config.password,
        database = db_name
    )
    cursor = cnx.cursor(buffered=True)
    
    cursor.execute("""
    CREATE TABLE {} (
    movie_id varchar(20) NOT NULL UNIQUE,
    imdb_id varchar(20) NOT NULL UNIQUE,
    title varchar(100) NOT NULL,
    budget int(12),
    revenue int(12),
    roi float(6),
    release_date varchar(10),
    PRIMARY KEY (movie_id)
    )
""".format(table_name))
    
    cnx.close()

In [8]:
create_tmdb_table('tmdb_comedy', db_name)

# DEFINE FUNCTIONS

"get_movie_ids" makes API calls to pull movie_ids (needed for reveneue/budget API calls). Takes a genre_id as a variable. Uses api_key defined above. Run time is approximately 30 seconds.

In [9]:
def get_movie_ids(genre_id):
    url = 'https://api.themoviedb.org/3/discover/movie?api_key={}&with_genres={}&sort_by=revenue.desc&page=1'.format(api_key, genre_id)
    response = requests.get(url)
    data = response.json()
    num = data['total_pages']
    
    movies = []
    page = 1
    while page < 100 or page <= num:
        url = 'https://api.themoviedb.org/3/discover/movie?api_key={}&with_genres={}&sort_by=revenue.desc&page={}'.format(api_key, genre_id, page)
        response = requests.get(url)
        moviez = response.json()['results']
        movies.extend(moviez)
        page += 1
        time.sleep(0.25)
        
    movie_ids = [movie['id'] for movie in movies]
    return movie_ids

"get_RevBub" makes one API call per movie_id and grabs revenue and budget infomation. Takes a list of movie_ids as a variable. Uses api_key defined above. Run time is approximately 6 minutes.

In [10]:
def get_RevBud(movie_ids):
    movies = []
    count = 1
    for i_d in movie_ids:
        url = 'https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US&append_to_response=revenue'.format(i_d,api_key)
        response = requests.get(url)
        moviez = response.json()
        movies.append(moviez)
        time.sleep(0.3)
    return movies

"parsed_results" returns cleaned results: imdb_id, title, budget, revenue, roi, release_date. Takes a list of movies (one dictionary per movie) as a variable.

In [11]:
def parsed_results(movies):
    parsed_movies = []
    for movie in movies:
        if movie['imdb_id'] == None or movie['imdb_id'] == '':
            continue
        else:
            parsed = {}
            parsed['movie_id'] = movie['id']
            parsed['imdb_id'] = movie['imdb_id']
            parsed['title'] = movie['title']
            parsed['budget'] = movie['budget']
            parsed['revenue'] = movie['revenue']
            parsed['release_date'] = movie['release_date']
            parsed_movies.append(parsed)
    return parsed_movies

"calc_roi" returns ROI = revenue / budget - 1. Takes a list of movies (one dictionary per movie) as a variable.

In [12]:
def calc_roi(parsed_movies):    
    for movie in parsed_movies:
        if movie['budget'] != 0:
            movie['roi'] = round((movie['revenue'] / movie['budget'])-1, 2)
        else:
            movie['roi'] = 0.0
    return parsed_movies

In [13]:
def insert_into_DB(table_name, movies):
    cnx = mysql.connector.connect(
        host = test_config.host,
        user = test_config.user,
        password = test_config.password,
        database = db_name
    )
    cursor = cnx.cursor(buffered=True)
    
    movies_to_insert = []
    for movie in movies:
        movie_tuple = (
            movie['movie_id'],
            movie['imdb_id'],
            movie['title'],
            movie['budget'],
            movie['revenue'],
            movie['roi'],
            movie['release_date']
        )
        movies_to_insert.append(movie_tuple)
        
    insert_query = ("""
        INSERT INTO {} (movie_id, imdb_id, title, budget, revenue, roi, release_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """.format(table_name))
    
    cursor.executemany(insert_query, movies_to_insert)
    cnx.commit()
    cnx.close()

In [15]:
def execute(genre_id, table_name):
    movie_ids = get_movie_ids(str(genre_id))
    movies = get_RevBud(movie_ids)
    parsed_movies = parsed_results(movies)
    movie_data = calc_roi(parsed_movies)
    insert_into_DB(table_name, parsed_movies)

In [16]:
execute(35, 'tmdb_comedy')

KeyboardInterrupt: 